In [920]:
import pandas as pd
import numpy as np

In [921]:
# load CSVs
School_data = "raw_data\schools_complete.csv"
Student_data = "raw_data\students_complete.csv"

In [922]:
# Read with pandas--low_memory required to suppress errors about mixed data types
Student_data_df = pd.read_csv(Student_data, low_memory=False)
#Student_data_df.head()

In [923]:
# Read with pandas--low_memory required to suppress errors about mixed data types
School_data_df = pd.read_csv(School_data, low_memory=False)
#School_data_df.head()
#School_data_df.columns

In [924]:
total_students = School_data_df["size"].sum()
#total_students

In [925]:
total_schools = School_data_df["name"].count()
#total_schools

In [926]:
avg_read_score = Student_data_df["reading_score"].mean()
avg_math_score = Student_data_df["math_score"].mean()
#print(avg_read_score)
#print(avg_math_score)

In [927]:
total_budget = School_data_df["budget"].sum()
#total_budget

In [928]:
readpass = Student_data_df[Student_data_df["reading_score"] >70]["reading_score"].count()
readpasspercent = readpass/total_students * 100
#readpasspercent

In [929]:
mathpass = Student_data_df[Student_data_df["math_score"] >70]["math_score"].count()
mathpasspercent = mathpass/total_students * 100
#mathpasspercent

In [930]:
# Create a new table consolodating above calculations
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [avg_math_score],
                                   "Average Reading Score": [avg_read_score],
                                   "% Passing Math":[mathpasspercent],
                                   "% Passing Reading":[readpasspercent],
                                   "% Overall Passing Rate": [overallpasspercent],
})
district_summary  = district_summary [["Total Schools", 
                                         "Total Students", 
                                         "Total Budget", 
                                         "Average Math Score", 
                                         "Average Reading Score", 
                                         "% Passing Math", 
                                         "% Passing Reading",
                                          "% Overall Passing Rate"]]
district_summary = district_summary.round(2)

#district_summary

In [931]:
overallpass_df = Student_data_df.loc[(Student_data_df["reading_score"] > 70) & (Student_data_df["math_score"] > 70), :]
overallpass = overallpass_df["Student ID"].count()
overallpasspercent = overallpass/total_students * 100
#overallpasspercent

In [932]:
School_data_df["Per Student Budget"] = School_data_df["budget"]/School_data_df["size"]
#School_data_df

In [933]:
district_summary["Total Students"] = district_summary["Total Students"].map("{0:,.0f}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${0:,.0f}".format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,72.39,82.97,60.8


In [934]:
Student_score_byschool = Student_data_df.groupby("school")
Student_score_byschool = Student_score_byschool["reading_score","math_score"].mean()
#Student_score_byschool

In [935]:
Student_readpass = Student_data_df.loc[Student_data_df["reading_score"] > 70,:]
Student_mathpass = Student_data_df.loc[Student_data_df["math_score"] > 70,:]
Student_overallpass = Student_data_df.loc[(Student_data_df["math_score"] > 70) & (Student_data_df["reading_score"] > 70) ,:]


In [936]:
Student_score_byschool["readpassno"] = Student_readpass.groupby("school")["reading_score"].count()
Student_score_byschool["mathpassno"] = Student_mathpass.groupby("school")["math_score"].count()
Student_score_byschool["overallpassno"] = Student_overallpass.groupby("school")["math_score"].count()
Student_score_byschool=Student_score_byschool.rename(columns={"reading_score":"Average Reading Score","math_score": "Average Math Score"})
#Student_score_byschool

In [937]:
School_data_df = School_data_df.rename(columns={"name":"school", "type":"School Type","size": "Total Students","budget": "Total School Budget"})
#School_data_df

In [938]:
School_data_df = School_data_df.loc[:,["school", "School Type","Total Students","Total School Budget","Per Student Budget"]]
#School_data_df

In [939]:
School_data_df=School_data_df.sort_values("school")
School_data_df= School_data_df.set_index("school")
#School_data_df 

In [941]:
school_summary_df = pd.merge(School_data_df, Student_score_byschool, left_index = True ,right_index = True)
#school_summary_df

In [942]:
school_summary_df["% Passing Math"] = school_summary_df["mathpassno"]/school_summary_df["Total Students"]*100
school_summary_df["% Passing Reading"] = school_summary_df["readpassno"]/school_summary_df["Total Students"]*100
school_summary_df["% Overall Passing Rate"] = school_summary_df["overallpassno"]/school_summary_df["Total Students"]*100
#school_summary_df.columns
#school_summary_df

In [943]:
school_summary = school_summary_df[['School Type', 'Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Reading Score', 'Average Math Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
#school_summary

In [944]:
school_summary_df = school_summary_df.drop('readpassno',axis=1)
school_summary_df = school_summary_df.drop('mathpassno',axis=1)
school_summary_df = school_summary_df.drop('overallpassno',axis=1)
#school_summary_df.columns

In [945]:
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${0:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${0:,.2f}".format)
school_summary = school_summary_df
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,81.033963,77.048432,64.630225,79.300643,51.145498
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,89.558665,93.864370,84.015070
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,63.750424,78.433367,49.915226
Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,65.753925,77.510040,51.296093
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,89.713896,93.392371,83.651226
Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,64.746494,78.187702,50.161812
Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,90.632319,92.740047,84.074941
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,63.318478,78.813850,49.914296
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,63.852132,78.281874,49.800462


In [946]:
top_performing_schools_bypassrate = school_summaryf.sort_values("% Overall Passing Rate", ascending=False)
top_performing_schools_bypassrate = top_performing_schools_bypassrate.iloc[0:4,0:9]
top_performing_schools_bypassrate

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,90.932983,93.254490,84.888305
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,91.683992,92.203742,84.823285
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.955000,83.682222,90.277778,93.444444,84.444444
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.848930,83.418349,90.214067,92.905199,84.281346


In [947]:
bottom_performing_schools_bypassrate = school_summaryf.sort_values("% Overall Passing Rate", ascending=True)
bottom_performing_schools_bypassrate = bottom_performing_schools_bypassrate.iloc[0:4,0:9]
bottom_performing_schools_bypassrate

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,64.066017,77.744436,49.437359
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,63.852132,78.281874,49.800462
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,63.318478,78.813850,49.914296
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,63.750424,78.433367,49.915226


In [948]:
Student_data_df["grade"] = Student_data_df["grade"].astype('category', categories = ["9th", "10th","11th","12th"])
pd.pivot_table(Student_data_df,index=["school"],values=["math_score"],
               columns=["grade"],aggfunc=[np.mean])

mean                                 
                      math_score                                 
grade                        9th       10th       11th       12th
school                                                           
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

In [949]:
pd.pivot_table(Student_data_df,index=["school"],values=["reading_score"],
               columns=["grade"],aggfunc=[np.mean])

mean                                 
                      reading_score                                 
grade                           9th       10th       11th       12th
school                                                              
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [950]:
school_summary_df["Per Student Budget"]=school_summary_df["Per Student Budget"].str.replace('$','')
school_summary_df["Per Student Budget"]=school_summary_df["Per Student Budget"].astype(float)
#school_summary_df

In [951]:
# Create the bins in which Data will be held
bins = [0, 585, 615, 645, 675]

# Create the names for the four bins
spend_ranges_group = ["<$585", '$585-615', '$615-645', '$645-675']

In [952]:
budget_df = school_summary_df

In [953]:
budget_df["Spending Ranges (Per Student)"] = pd.cut(budget_df["Per Student Budget"], bins, labels=spend_ranges_group)
#budget_df

In [954]:
school_summary_df_sr = budget_df.groupby("Spending Ranges (Per Student)")
school_summary_df_sr.max()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,,,,,
<$585,Charter,2283,"$248,087.00",583.0,83.989488,83.803279,90.932983,93.864370,84.888305
$585-615,Charter,1761,"$585,858.00",609.0,84.044699,83.839917,91.683992,92.617831,84.823285
$615-645,District,4976,"$917,500.00",644.0,83.848930,83.418349,90.214067,93.392371,84.281346
$645-675,District,4761,"$3,094,650.00",655.0,81.182722,77.289752,64.746494,78.813850,50.161812


In [955]:
# Create the bins in which Data will be held
schoolsizebins = [0, 1000, 2000, 5000]

# Create the names for the four bins
school_size_group = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
school_summary_bysize = school_summary
#school_summary_bysize

In [956]:
school_summary_bysize["Size Range"] = pd.cut(school_summary_bysize["Total Students"], schoolsizebins, labels=school_size_group)
#school_summary_bysize

In [957]:
school_summary_dfbysize = school_summary_bysize.groupby("Size Range")
school_summary_dfbysize.max()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student)
Size Range,,,,,,,,,,
Small (<1000),Charter,962,"$585,858.00",609.0,84.044699,83.839917,91.683992,92.740047,84.823285,$585-615
Medium (1000-2000),Charter,1858,"$917,500.00",638.0,83.975780,83.682222,90.277778,93.864370,84.444444,$615-645
Large (2000-5000),District,4976,"$3,124,928.00",655.0,83.989488,83.274201,90.932983,93.254490,84.888305,$645-675


In [958]:
school_summary_bytype = school_summary_df.groupby("School Type")
school_summary_bytype.max()

,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student),Size Range
School Type,,,,,,,,,,
Charter,2283,"$917,500.00",638.0,84.044699,83.839917,91.683992,93.864370,84.888305,$615-645,Large (2000-5000)
District,4976,"$3,124,928.00",655.0,81.182722,77.289752,65.753925,79.300643,51.296093,$645-675,Large (2000-5000)
